In [3]:
import xgboost as xgb
import sklearn.datasets  
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb

import re
import shap
import numpy as np
# Load a sample dataset
X, y = sklearn.datasets.load_iris(return_X_y=True)
data= sklearn.datasets.load_iris()

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_data = lgb.Dataset(X_train, label=y_train)

params = {
    'objective': 'multiclass',    # Objective for classification
    'num_class': 3,               # Number of classes in the target
    'metric': 'multi_logloss',    # Metric for evaluation
    'boosting_type': 'gbdt',      # Gradient Boosting Decision Tree
    'learning_rate': 0.1,         # Learning rate
    'num_leaves': 31,             # Number of leaves in one tree
    'max_depth': -1,              # Unlimited depth of trees
    'verbose': -1                 # Suppress output
}
# Train an XGBoost classifier
model = lgb.train(params, train_data, num_boost_round=100)



In [6]:
import lightgbm as lgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd

# Load the Iris dataset
data = load_iris()
X, y = data.data, data.target

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LightGBM dataset from the training data
train_data = lgb.Dataset(X_train, label=y_train)

# Set up parameters for the LightGBM model
params = {
    'objective': 'multiclass',
    'num_class': 3,
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'learning_rate': 0.1,
    'num_leaves': 31,
    'max_depth': -1,
    'verbose': -1
}

# Train the LightGBM model
model = lgb.train(params, train_data, num_boost_round=10)

# Dump the model into a Python dictionary
model_dump = model.dump_model()

# Extract tree data in a structured format
tree_structure = model_dump['tree_info']
tree_data = []

# Extracting data from the tree info for each node
for tree_index, tree in enumerate(tree_structure):
    nodes = tree['tree_structure']
    
    def extract_nodes(node, tree_index):
        if 'split_index' in node:
            # Internal node
            tree_data.append({
                'Tree': tree_index,
                'Node': node['split_index'],
                'Feature': node['split_feature'],
                'Threshold': node['threshold'],
                'Decision_Type': node['decision_type'],
                'Gain': node.get('split_gain', None),
                'Left_Child': node['left_child']['split_index'] if 'left_child' in node else None,
                'Right_Child': node['right_child']['split_index'] if 'right_child' in node else None,
            })
            # Recursively extract child nodes
            if 'left_child' in node:
                extract_nodes(node['left_child'], tree_index)
            if 'right_child' in node:
                extract_nodes(node['right_child'], tree_index)
        else:
            # Leaf node
            tree_data.append({
                'Tree': tree_index,
                'Node': node.get('leaf_index', None),
                'Feature': 'Leaf',
                'Threshold': None,
                'Decision_Type': None,
                'Gain': None,
                'Left_Child': None,
                'Right_Child': None,
            })
    
    # Start extraction from the root node
    extract_nodes(nodes, tree_index)

# Convert the tree data into a DataFrame
df = pd.DataFrame(tree_data)

# Display the DataFrame
print(df.head())


KeyError: 'split_index'

In [52]:
BC = sklearn.datasets.load_iris()
df = pd.DataFrame(data=BC.data, columns=BC.feature_names)
df['target'] = BC.target

In [54]:
max_val_dict = df.describe().loc['max']
min_val_dict = df.describe().loc['min']

In [55]:
max_val_dict

sepal length (cm)    7.9
sepal width (cm)     4.4
petal length (cm)    6.9
petal width (cm)     2.5
target               2.0
Name: max, dtype: float64

In [56]:
min_val_dict

sepal length (cm)    4.3
sepal width (cm)     2.0
petal length (cm)    1.0
petal width (cm)     0.1
target               0.0
Name: min, dtype: float64

In [57]:
explainer = shap.Explainer(model)

# Calculate SHAP values
shap_values = explainer(X_train)

In [58]:
# feature_importances = pd.DataFrame({
#     'Feature': data.feature_names,
#     'Importance': np.abs(shap_values.values).mean(axis=0)
# })

# # Sort features by importance
# feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

In [60]:
#feature_importances

In [61]:
# Convert the model trees to a DataFrame
booster = model.get_booster()
trees_df = booster.trees_to_dataframe()


In [62]:
def switch(feature_names, series):
    # Create a mapping dictionary
    feature_map = {f'f{i+1}': name for i, name in enumerate(feature_names)}
    # Map the series using the dictionary
    return series.map(feature_map)

In [63]:
trees_df['Feature1'] = switch(data['feature_names'], trees_df['Feature'])

In [64]:
trees_df['Feature1'] = trees_df['Feature1'].str.replace(" ", "_", regex=False)
trees_df['Feature1'] = trees_df['Feature1'].fillna('Leaf')
trees_df.head(20)

Tree  Node   ID Feature  Split  Yes   No Missing       Gain      Cover  \
0      0     0  0-0      f2    3.0  0-1  0-2     0-2  46.247253  40.888886   
1      0     1  0-1    Leaf    NaN  NaN  NaN     NaN   0.015000  13.777777   
2      0     2  0-2    Leaf    NaN  NaN  NaN     NaN  -0.007500  27.111109   
3      1     0  1-0      f2    3.0  1-1  1-2     1-2  45.957420  41.777775   
4      1     1  1-1    Leaf    NaN  NaN  NaN     NaN   0.015000  13.333332   
5      1     2  1-2    Leaf    NaN  NaN  NaN     NaN  -0.007500  28.444443   
6      2     0  2-0      f2    3.0  2-1  2-2     2-2  46.747315  41.777775   
7      2     1  2-1    Leaf    NaN  NaN  NaN     NaN   0.015000  13.777777   
8      2     2  2-2    Leaf    NaN  NaN  NaN     NaN  -0.007500  27.999998   
9      3     0  3-0      f3    1.0  3-1  3-2     3-2  45.988979  40.444443   
10     3     1  3-1    Leaf    NaN  NaN  NaN     NaN   0.015000  13.777777   
11     3     2  3-2    Leaf    NaN  NaN  NaN     NaN  -0.007500  26.666664   
12     4     0  4-0      f2    3.0  4-1  4-2     4-2  43.840176  43.111107   
13     4     1  4-1    Leaf    NaN  NaN  NaN     NaN   0.015000  11.999999   
14     4     2  4-2    Leaf    NaN  NaN  NaN     NaN  -0.007500  31.111109   
15     5     0  5-0      f2    3.0  5-1  5-2     5-2  48.726536  42.666664   
16     5     1  5-1    Leaf    NaN  NaN  NaN     NaN   0.015000  14.666666   
17     5     2  5-2    Leaf    NaN  NaN  NaN     NaN  -0.007500  27.999998   
18     6     0  6-0      f2    3.0  6-1  6-2     6-2  50.747494  44.888886   
19     6     1  6-1    Leaf    NaN  NaN  NaN     NaN   0.015000  15.111111   

    Category           Feature1  
0        NaN   sepal_width_(cm)  
1        NaN               Leaf  
2        NaN               Leaf  
3        NaN   sepal_width_(cm)  
4        NaN               Leaf  
5        NaN               Leaf  
6        NaN   sepal_width_(cm)  
7        NaN               Leaf  
8        NaN               Leaf  
9        NaN  petal_length_(cm)  
10       NaN               Leaf  
11       NaN               Leaf  
12       NaN   sepal_width_(cm)  
13       NaN               Leaf  
14       NaN               Leaf  
15       NaN   sepal_width_(cm)  
16       NaN               Leaf  
17       NaN               Leaf  
18       NaN   sepal_width_(cm)  
19       NaN               Leaf

In [65]:
def get_rules(tree_no, index, leaf_id):
    # Initialize the list to store the rule path
    rules = []
    
    # Start with the leaf node
    current_node = trees_df[trees_df['ID'] == leaf_id]
    
    while not current_node.empty:
        feature = current_node['Feature1'].values[0]
        split = current_node['Split'].values[0]
        parent_id = current_node['Node'].values[0]
        # Determine if the current node is a 'Yes' or 'No' branch from its parent
        parent_node = trees_df[trees_df['Yes'] == leaf_id] if not trees_df[trees_df['Yes'] == leaf_id].empty else trees_df[trees_df['No'] == leaf_id]
        if parent_node.empty:
            break
        
        parent_feature = parent_node['Feature1'].values[0]
        parent_split = parent_node['Split'].values[0]
        
        if leaf_id in parent_node['Yes'].values:
            rule = f"{parent_feature} <= {parent_split}"
        else:
            rule = f"{parent_feature} > {parent_split}"
        
        rules.append(rule)
        
        # Move to the parent node
        leaf_id = parent_node['ID'].values[0]
        current_node = parent_node
    
    # Reverse the rules to get the correct order from root to leaf
    rules.reverse()
    
    # Print the rule path and the class prediction at the leaf
    #predicted_class = 1 if current_node['Gain'].values[0] > 0 else 0
    rule_path = " and ".join(rules)
    return(f"Tree {tree_no} - Rule {index}: {rule_path} -> {current_node['Gain'].values[0]}")


In [66]:
ruleset=[]
for tree_no in range(100):

    current_tree = trees_df[trees_df['Tree']==tree_no]

    leaf_nodes = current_tree[current_tree['Feature1'] == 'Leaf']

    # Get IDs of leaf nodes
    leaf_ids = leaf_nodes['ID'].tolist()
    for index,id in enumerate(leaf_ids):
        rule = get_rules(tree_no, index, id)
        ruleset.append(rule)
        #print(rule)


In [67]:
tree_no =1
current_tree = trees_df[trees_df['Tree']==tree_no]

leaf_nodes = current_tree[current_tree['Feature1'] == 'Leaf']

# Get IDs of leaf nodes
leaf_ids = leaf_nodes['ID'].tolist()

for index,id in enumerate(leaf_ids):
        rule = get_rules(tree_no, index, id)
        ruleset.append(rule)
        print(rule)

Tree 1 - Rule 0: sepal_width_(cm) <= 3.0 -> 45.9574203
Tree 1 - Rule 1: sepal_width_(cm) > 3.0 -> 45.9574203


In [70]:
new= []
for i in range(len(X_train)):
    if df.iloc[i]['sepal length (cm)']> 3:
        new.append(i)

In [71]:
new

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119]

In [50]:
min_val_dict

sepal length (cm)    4.3
sepal width (cm)     2.0
petal length (cm)    1.0
petal width (cm)     0.1
target               0.0
Name: min, dtype: float64

##Creating Ruleset


In [15]:
def condition_extraction(ruleset):
    # Pattern to match everything after the colon and before the arrow
    pattern_initial = r'(?<=:\s)(.*?)(?=\s->)'
    
    # Find all matches
    matches = re.findall(pattern_initial, ruleset)

    conditions = []
    if matches:
        # Extract the first match which contains all conditions
        condition_string = matches[0]
        
        # Pattern to separate conditions by 'and'
        pattern = r'\s+and\s+'
        
        # Split the condition string using 'and' keyword
        conditions = re.split(pattern, condition_string)
        
        # Strip any leading or trailing spaces
        conditions = [cond.strip() for cond in conditions]

    return conditions

In [17]:
def check_conditions(data_point_dict, conditions):
    extracted_conditions=  condition_extraction(conditions)
    for condition in extracted_conditions:

        pattern_for_feature =r'[A-Za-z_]+'
        feature_match = re.search(pattern_for_feature, condition)
        
        pattern_for_symbol = r'[<>=]+'
        symbol_match = re.search(pattern_for_symbol, condition)
        
        pattern_for_value = r'[-+]?\d*\.?\d+'
        value_match = re.search(pattern_for_value, condition)
        
        if feature_match and symbol_match and value_match:
            # Extract the matched groups
            fea=[]
            feature = feature_match.group()
            symbol = symbol_match.group()
            value = (value_match.group())
            feature = feature.split('_')
            for i in range(len(feature)):
                fea.append(f'{feature[i]} ')
            feature = "".join(fea)
            if feature[:-1] not in data_point_dict:
                continue
            feature_value = data_point_dict[feature[:-1]]
            expression = f"{feature_value} {symbol} {value}"
            if not eval(expression): 
                return False
    return True

In [18]:
def get_neigbors(ruleset, data_points):
    neighbors = []
    counter = 0
    for index in range(len(data_points)):
        dictionary = dict(df.iloc[index].squeeze())
        check = check_conditions(dictionary,ruleset)
        #print(check, index)
        if check == True:
            neighbors.append(index)
            counter +=1
            #print(counter)
            
    return neighbors

In [19]:
leaf_neighborhoods = []
for rule in ruleset:    
    leaf_neighborhoods.append((get_neigbors(rule,df)))
    

In [20]:
leaf_neighborhoods_counts = [len(neighbor) for neighbor in leaf_neighborhoods]

In [31]:
len(leaf_neighborhoods[6])

150

In [22]:
tree_number = int(ruleset[0].split(' ')[1])
tree_number

0

In [23]:
neighbors_in_one_tree =[None] * len(ruleset)
counter =0
for rule in ruleset:
    tree_number = int(rule.split(' ')[1])
    #print(tree_number)

    if not tree_number == counter:
        counter += 1
    neighbors_in_one_tree[counter]=(get_neigbors(rule, df))

In [25]:
ruleset[1]

'Tree 0 - Rule 1: sepal_width_(cm) > 3.0 -> 57.3227081'